<a href="https://colab.research.google.com/github/guymatz/csci-49376/blob/develop/csci_49376_project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
from google.colab import drive
drive.mount('/content/drive')
#drive.flush_and_unmount()

Mounted at /content/drive


In [29]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=ae7a4c1ab92e2ee3a69818a9969541a059777f98d09d214c4b9750498e71cfd0
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [30]:
# Import SparkSession
from pyspark.sql import SparkSession
from pyspark.sql import functions
# Create a Spark Session
ss = SparkSession.builder.master("local[*]").getOrCreate()

In [48]:
edges = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/edges.tsv", header=True, sep="\t")
nodes = ss.read.csv("/content/drive/MyDrive/School/CSCI/Big-data/nodes.tsv", header=True, sep="\t")

In [49]:
print(f"Edges: {edges.count()}")
print(f"Nodes: {nodes.count()}")

Edges: 1292203
Nodes: 23036


In [50]:
# For each drug, compute the number of genes
# and the number of diseases associated with the
# drug. Output results with top 5 number of genes in a
# descending order

In [51]:
compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Gene') 
        |
        edges.target.startswith('Disease')
    )
)

In [52]:
compounds.sample(.01).show()

+-----------------+--------+-------------------+
|           source|metaedge|             target|
+-----------------+--------+-------------------+
|Compound::DB00860|     CtD| Disease::DOID:3083|
|Compound::DB00178|     CtD|Disease::DOID:10763|
|Compound::DB00688|     CtD| Disease::DOID:9074|
|Compound::DB00620|     CtD| Disease::DOID:7148|
|Compound::DB01174|     CtD| Disease::DOID:1826|
|Compound::DB00273|     CtD| Disease::DOID:6364|
|Compound::DB00570|     CbG|        Gene::10382|
|Compound::DB01088|     CbG|         Gene::5142|
|Compound::DB05294|     CbG|         Gene::8767|
|Compound::DB00665|     CbG|         Gene::1558|
|Compound::DB00841|     CbG|          Gene::154|
|Compound::DB01166|     CbG|         Gene::1551|
|Compound::DB00163|     CbG|         Gene::2947|
|Compound::DB00882|     CbG|        Gene::10280|
|Compound::DB01201|     CbG|        Gene::64816|
|Compound::DB00334|     CbG|         Gene::3351|
|Compound::DB00943|     CbG|         Gene::9356|
|Compound::DB00149| 

In [54]:
filtered_compounds = compounds.rdd.map(lambda x: ((x.source,"G"), 1) if x.target.startswith("Gene") else ((x.source, "D"), 1))
filtered_compounds.take(10)

[(('Compound::DB00997', 'D'), 1),
 (('Compound::DB00206', 'D'), 1),
 (('Compound::DB00960', 'D'), 1),
 (('Compound::DB00665', 'D'), 1),
 (('Compound::DB00290', 'D'), 1),
 (('Compound::DB01232', 'D'), 1),
 (('Compound::DB00555', 'D'), 1),
 (('Compound::DB00444', 'D'), 1),
 (('Compound::DB00860', 'D'), 1),
 (('Compound::DB00635', 'D'), 1)]

In [55]:
filtered_compound_counts = filtered_compounds.reduceByKey(lambda a,b:a + b)
filtered_compound_counts.take(5)

[(('Compound::DB00960', 'D'), 1),
 (('Compound::DB00665', 'D'), 1),
 (('Compound::DB01232', 'D'), 1),
 (('Compound::DB00555', 'D'), 3),
 (('Compound::DB00635', 'D'), 17)]

In [56]:
sorted_count = filtered_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_count.take(5)

[(('Compound::DB08865', 'G'), 585),
 (('Compound::DB01254', 'G'), 564),
 (('Compound::DB00997', 'G'), 532),
 (('Compound::DB00570', 'G'), 523),
 (('Compound::DB00390', 'G'), 522)]

In [ ]:
# Q2: Compute the number of diseases associated
# with 1, 2, 3, …, n drugs. Output results with the top
# 5 number of diseases in a descending order.
disease_compounds = edges.filter(
    edges.source.startswith('Compound') &
    (
        edges.target.startswith('Disease')
    )

)

In [ ]:
filtered_disease_compounds = disease_compounds.rdd.map(lambda x: (x.target,1))
filtered_disease_compounds.take(5)
filtered_disease_compound_counts = filtered_disease_compounds.reduceByKey(lambda a,b:a + b)
sorted_disease_count = filtered_disease_compound_counts.sortBy(lambda x: x[1], ascending=False)
sorted_disease_count.take(5)

[('Disease::DOID:10763', 70),
 ('Disease::DOID:2531', 53),
 ('Disease::DOID:1612', 43),
 ('Disease::DOID:2841', 40),
 ('Disease::DOID:3393', 38)]

In [ ]:
# Q3: Get the name of drugs that have the top 5 number of genes. Output the results.